## Official documentation

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

chain.invoke({"topic": "ice cream"})

"Why don't ice cream cones have secrets?\n\nBecause they always spill the scoop!"

Below are the internal workflow flow (prompt->model->output_parser) when chain.invoke({"topic": "ice cream"}) is called:

prompt: BasePromptTemplate.invoke(input, config): This method is called first, with input being {"topic": "ice cream"}. It formats the prompt using the input variables and returns a PromptValue object. This object represents a filled prompt that's ready to be passed to a language model.

model: BaseChatModel.invoke(input, config, **kwargs): This method is called next, with input being the PromptValue object returned by the previous step. It converts the PromptValue object into a format that's suitable for the language model, generates a response using the language model, and returns a BaseMessage object. This object represents the response generated by the language model.

output_parser: BaseGenerationOutputParser.invoke(input, config): This method is called last, with input being the BaseMessage object returned by the previous step. It parses the BaseMessage object and returns the parsed result.

Each invoke method is responsible for handling one step in the pipeline, and the output of each step is passed as the input to the next step. This allows for a clear separation of responsibilities and makes it easy to modify or replace individual steps in the pipeline.

# [MermaidChart: 15383246-018f-44ee-9732-71ffd9e6c924]

In [10]:
prompt_value = prompt.invoke({"topic": "ice cream"})
print('prompt_value: ', prompt_value)
print('prompt_value.to_string(): ', prompt_value.to_string())
print('prompt_value.to_messages(): ', prompt_value.to_messages())
print('prompt_value.to_json(): ', prompt_value.to_json())

prompt_value:  messages=[HumanMessage(content='tell me a short joke about ice cream')]
prompt_value.to_string():  Human: tell me a short joke about ice cream
prompt_value.to_messages():  [HumanMessage(content='tell me a short joke about ice cream')]
prompt_value.to_json():  {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'prompts', 'chat', 'ChatPromptValue'], 'kwargs': {'messages': [HumanMessage(content='tell me a short joke about ice cream')]}}


In [18]:
# from langchain_openai import ChatOpenAI
# model = ChatOpenAI(model="gpt-4")
message  = model.invoke(prompt_value)
print('ChatOpenAI message: ', message)

from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
print('OpenAI message: ', llm.invoke(prompt_value))

ChatOpenAI message:  content="Why don't ice creams ever get invited to parties?\n\nBecause they always drip when they're in the heat!"
OpenAI message:  

Robot: Why did the ice cream go to therapy? Because it was having a meltdown!


In [21]:
result = output_parser.invoke(message)
print('result: ', result)

result:  Why don't ice creams ever get invited to parties?

Because they always drip when they're in the heat!


In [2]:
# Requires:
# pip install langchain docarray tiktoken
# Fix the error in community lib according to PR: https://github.com/langchain-ai/langchain/pull/18678 befor executing this code
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")

/home/ubuntu/anaconda3/envs/py310/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


'Harrison worked at Kensho.'